# The Graph Data Access

In this notebook, we retrieve live Rai system data from [The Graph](https://thegraph.com/explorer/subgraph/reflexer-labs/rai-mainnet?query=Example%20query) and save it as a CSV for analysis in the [Graph Analysis](TheGraphDataAnalysis.ipynb) notebook.

In [1]:
# import libraries
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

## Hourly stats

In [2]:
query_header = '''
query {{
    hourlyStats(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
 timestamp
    blockNumber
    redemptionPrice {
      id
      block
      redemptionRate
      value
      
    }
    marketPriceUsd
    marketPriceEth
    globalDebt
    erc20CoinTotalSupply
'''





n = 0
safes = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats']
    print(f'results {len(s)}')
    safes.extend(s)
    print(f'total {len(safes)}')
    n += 1
    if len(s) < 1000:
        break
        
hourlyStats = pd.DataFrame(safes)

request 1
results 533
total 533


In [3]:
hourlyStats

,blockNumber,erc20CoinTotalSupply,globalDebt,marketPriceEth,marketPriceUsd,redemptionPrice,timestamp
0,11857308,1988.440432971667032725,1988.577499489843641169436691184972,0.002589462210048543650949087456484599,4.670500166535429740591356399938391,"{'block': '11849022', 'id': '0xe313271188492c4...",1613338681
1,11860793,2788.440432971667032725,2788.617717345781163523077653358556,0.002481355746535171105569862943787003,4.377049344386119132419461918241039,"{'block': '11860755', 'id': '0x40066774f6fb7d3...",1613384616
2,11861040,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.406833024856393396658172078078713,"{'block': '11861008', 'id': '0x0d551e2a2d55fa8...",1613387890
3,11861337,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,"{'block': '11861237', 'id': '0x5c67dbb323db039...",1613392101
4,11861474,3048.440432971667032725,3048.645954953510904626288422086589,0.001768934955701819449591545208406114,3.194329891923853932595347583210722,"{'block': '11861237', 'id': '0x5c67dbb323db039...",1613394096
...,...,...,...,...,...,...,...
528,12004508,38184776.168264137934523732,39055774.24393665748716824706357697,0.001685067200812686660325687448650719,3.041762829849153580725105824292708,"{'block': '12004225', 'id': '0x34689642cccc0ee...",1615295876
529,12004773,38183776.168264137934523732,39054799.54730119653910667257414807,0.001687546593321897396681132906493676,3.046238451932051143883122495343608,"{'block': '12004724', 'id': '0x09d2edad16b9237...",1615299608
530,12004991,38112130.293653287875410629,38983232.58322681413246348008142746,0.001692950691996243431739229282649061,3.094878922464244002525438368098108,"{'block': '12004724', 'id': '0x09d2edad16b9237...",1615302277
531,12005265,38243830.232160960017901484,39115026.9079094055918424427017571,0.001692894529056418722913695078570085,3.089651018144998118628097477045905,"{'block': '12005013', 'id': '0x36808ec089027b7...",1615305899


In [4]:
hourlyStats.redemptionPrice.values[0]

{'block': '11849022',
 'id': '0xe313271188492c4cb881c814c2ce2797f638c56c4f0198abbfc7da11d9b06ed3-7',
 'redemptionRate': '1',
 'value': '3.14'}

In [5]:
hourlyStats['redemptionPriceActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['value'])

In [6]:
hourlyStats['redemptionRateActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['redemptionRate'])

In [7]:
hourlyStats['erc20CoinTotalSupply'] = hourlyStats['erc20CoinTotalSupply'].astype(float)
hourlyStats['globalDebt'] = hourlyStats['globalDebt'].astype(float)
hourlyStats['blockNumber'] = hourlyStats['blockNumber'].astype(int)
hourlyStats['marketPriceEth'] = hourlyStats['marketPriceEth'].astype(float)
hourlyStats['marketPriceUsd'] = hourlyStats['marketPriceUsd'].astype(float)
hourlyStats['redemptionPriceActual'] = hourlyStats['redemptionPriceActual'].astype(float)
hourlyStats['redemptionRateActual'] = hourlyStats['redemptionRateActual'].astype(float)

In [8]:
hourlyStats.describe()

,blockNumber,erc20CoinTotalSupply,globalDebt,marketPriceEth,marketPriceUsd,redemptionPriceActual,redemptionRateActual
count,5.330000e+02,5.330000e+02,5.330000e+02,533.000000,533.000000,533.000000,5.330000e+02
mean,1.193335e+07,2.540810e+07,2.591990e+07,0.002013,3.348852,3.094425,1.000000e+00
std,4.187935e+04,1.241339e+07,1.276372e+07,0.000312,0.518664,0.040308,1.531488e-08
min,1.185731e+07,1.988440e+03,1.988577e+03,0.001609,2.850455,3.018131,9.999999e-01
25%,1.189726e+07,1.381372e+07,1.382187e+07,0.001798,3.168261,3.057628,1.000000e+00
50%,1.193332e+07,3.254064e+07,3.349321e+07,0.001982,3.235196,3.107139,1.000000e+00
75%,1.196947e+07,3.435062e+07,3.527843e+07,0.002122,3.331552,3.130402,1.000000e+00
max,1.200566e+07,3.832670e+07,3.919798e+07,0.004474,8.055382,3.140000,1.000000e+00


In [9]:
hourlyStats.to_csv('saved_results/RaiLiveGraphData.csv')

## Collateral Prices

## Safes

## Conclusion

Using The Graph, a lot of data about the Rai system can be obtained for analyzing the health of the system. With some data manipulation, these data streams could be intergrated into the Rai cadCAD model to turn it into a true decision support system.